In [4]:
from bs4 import BeautifulSoup
from splinter import Browser
from pprint import pprint
import pymongo
import pandas as pd
import requests
import re
import jellyfish
import numpy as np

In [2]:
clean_df = pd.read_csv("final_clean_a_to_z.csv")
clean_df

,books,authors,movies,released_year
0,"The A.B.C. Murders (1936),",Agatha Christie,The Alphabet Murders,1966.0
1,"The A.B.C. Murders (1936),",Agatha Christie,The A.B.C. Murders,1992.0
2,"À ton image (1998),",Louise L. Lambrichs,À ton image,2004.0
3,"De Aanslag (The Assault) (1982),",Harry Mulisch,The Assault,1986.0
4,"About a Boy (1998),",Nick Hornby,About a Boy,2002.0
...,...,...,...,...
2788,"The Story of Zarak Khan (1949),",A. J. Bevan,Zarak,1956.0
2789,"Zathura (2002),",Chris Van Allsburg,Zathura,2005.0
2790,"Zona Zamfirova (1906),",Stefan Sremac,Zona Zamfirova,2002.0
2791,"Zorba the Greek (1946),",Nikos Kazantzakis,Zorba the Greek,1964.0


In [31]:
movie = "The Alphabet Murders"
url ="https://en.wikipedia.org/wiki/" + movie
# create soup
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   The Alphabet Murders - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"9c28dd9c-c585-4058-a1a6-23fa375a070c","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"The_Alphabet_Murders","wgTitle":"The Alphabet Murders","wgCurRevisionId":978290404,"wgRevisionId":978290404,"wgArticleId":8262764,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Use dmy dates from June 2015","Use British English from June 2015","Template film date with 2 release dates","All stub articles","

In [138]:
#test
myuls = soup.find_all("ul")
for uls in myuls:
    link_section = uls.find("a", string = movie, href = True)
    try: 
        pprint(link_section["href"])
    except Exception:
        pass

'https://www.imdb.com/title/tt0060094/'


In [22]:
#scrape wikipedia for imdb links 
movies = clean_df["movies"].to_list()
imdb_links = []
test = ['The Alphabet Murders','The A.B.C. Murders', 'À ton image','The Assault','About a Boy']
for movie in movies:
    try: 
        url ="https://en.wikipedia.org/wiki/" + movie
        # create soup
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        try:
            myuls = soup.find_all("ul")
            for uls in myuls:
                link_section = uls.find("a", string = movie, href = True)
                if link_section:
                    imdb =  link_section["href"]
                    break
                else:
                    imdb = ""
        except Exception:
            pass 
    except Exception:
        pass
    imdb_links.append(imdb)
    print(imdb)

imdb_links

https://www.imdb.com/title/tt0060094/

https://www.imdb.com/title/tt0307343/


https://www.imdb.com/title/tt0257360/

/wiki/The_Accidental_Tourist_(film)
https://www.imdb.com/title/tt0068168/
https://www.imdb.com/title/tt0479341/



https://www.gutenberg.org/ebooks/13861
/wiki/Category:Adventure

https://www.imdb.com/title/tt0395119/
https://www.imdb.com/title/tt0046683/
https://www.imdb.com/title/tt0009209/
/wiki/Huckleberry_Finn_(1920_film)
/wiki/Huckleberry_Finn_(1920_film)

/wiki/The_Adventures_of_Huckleberry_Finn_(1939_film)
/wiki/The_Adventures_of_Huckleberry_Finn_(1939_film)
/wiki/The_Adventures_of_Huckleberry_Finn_(1939_film)
/wiki/Huckleberry_Finn_(1920_film)
https://www.allmovie.com/movie/v155718
/wiki/Huckleberry_Finn_(1920_film)
/wiki/The_Adventures_of_Huckleberry_Finn_(1939_film)
/w/index.php?title=Adventures_of_Huckleberry_Finn_(1985_TV_series)&action=edit&redlink=1

https://www.imdb.com/title/tt0099087/
/wiki/The_Adventures_of_Huck_Finn_(1993_film)
https://www.imdb.com/t

['https://www.imdb.com/title/tt0060094/',
 '',
 'https://www.imdb.com/title/tt0307343/',
 '',
 '',
 'https://www.imdb.com/title/tt0257360/',
 '',
 '/wiki/The_Accidental_Tourist_(film)',
 'https://www.imdb.com/title/tt0068168/',
 'https://www.imdb.com/title/tt0479341/',
 '',
 '',
 '',
 'https://www.gutenberg.org/ebooks/13861',
 '/wiki/Category:Adventure',
 '',
 'https://www.imdb.com/title/tt0395119/',
 'https://www.imdb.com/title/tt0046683/',
 'https://www.imdb.com/title/tt0009209/',
 '/wiki/Huckleberry_Finn_(1920_film)',
 '/wiki/Huckleberry_Finn_(1920_film)',
 '',
 '/wiki/The_Adventures_of_Huckleberry_Finn_(1939_film)',
 '/wiki/The_Adventures_of_Huckleberry_Finn_(1939_film)',
 '/wiki/The_Adventures_of_Huckleberry_Finn_(1939_film)',
 '/wiki/Huckleberry_Finn_(1920_film)',
 'https://www.allmovie.com/movie/v155718',
 '/wiki/Huckleberry_Finn_(1920_film)',
 '/wiki/The_Adventures_of_Huckleberry_Finn_(1939_film)',
 '/w/index.php?title=Adventures_of_Huckleberry_Finn_(1985_TV_series)&action=edit

In [23]:
clean_df["imdb_links"] = imdb_links

,books,authors,movies,released_year,imdb_links
0,"The A.B.C. Murders (1936),",Agatha Christie,The Alphabet Murders,1966.0,https://www.imdb.com/title/tt0060094/
1,"The A.B.C. Murders (1936),",Agatha Christie,The A.B.C. Murders,1992.0,
2,"À ton image (1998),",Louise L. Lambrichs,À ton image,2004.0,https://www.imdb.com/title/tt0307343/
3,"De Aanslag (The Assault) (1982),",Harry Mulisch,The Assault,1986.0,
4,"About a Boy (1998),",Nick Hornby,About a Boy,2002.0,
...,...,...,...,...,...
2788,"The Story of Zarak Khan (1949),",A. J. Bevan,Zarak,1956.0,https://www.imdb.com/title/tt0051219/
2789,"Zathura (2002),",Chris Van Allsburg,Zathura,2005.0,
2790,"Zona Zamfirova (1906),",Stefan Sremac,Zona Zamfirova,2002.0,https://www.imdb.com/title/tt0326449/
2791,"Zorba the Greek (1946),",Nikos Kazantzakis,Zorba the Greek,1964.0,/wiki/Zorba_the_Greek_(film)


In [24]:
#save to new file
clean_df.to_csv("a_to_z_with_links.csv", encoding="utf-8", index=False)
clean_df

,books,authors,movies,released_year,imdb_links
0,"The A.B.C. Murders (1936),",Agatha Christie,The Alphabet Murders,1966.0,https://www.imdb.com/title/tt0060094/
1,"The A.B.C. Murders (1936),",Agatha Christie,The A.B.C. Murders,1992.0,
2,"À ton image (1998),",Louise L. Lambrichs,À ton image,2004.0,https://www.imdb.com/title/tt0307343/
3,"De Aanslag (The Assault) (1982),",Harry Mulisch,The Assault,1986.0,
4,"About a Boy (1998),",Nick Hornby,About a Boy,2002.0,
...,...,...,...,...,...
2788,"The Story of Zarak Khan (1949),",A. J. Bevan,Zarak,1956.0,https://www.imdb.com/title/tt0051219/
2789,"Zathura (2002),",Chris Van Allsburg,Zathura,2005.0,
2790,"Zona Zamfirova (1906),",Stefan Sremac,Zona Zamfirova,2002.0,https://www.imdb.com/title/tt0326449/
2791,"Zorba the Greek (1946),",Nikos Kazantzakis,Zorba the Greek,1964.0,/wiki/Zorba_the_Greek_(film)


In [10]:
#get data with only imdb links
show = pd.read_csv("a_to_z_with_links.csv")
show = show.dropna()
show = show[show["imdb_links"].str.contains('imdb')]
show

,books,authors,movies,released_year,imdb_links
0,"The A.B.C. Murders (1936),",Agatha Christie,The Alphabet Murders,1966.0,https://www.imdb.com/title/tt0060094/
2,"À ton image (1998),",Louise L. Lambrichs,À ton image,2004.0,https://www.imdb.com/title/tt0307343/
5,"About Schmidt (1996),",Louis Begley,About Schmidt,2002.0,https://www.imdb.com/title/tt0257360/
8,"Across 110th (1970),",Wally Ferris,Across 110th Street,1972.0,https://www.imdb.com/title/tt0068168/
9,"Adam Resurrected (Man, Son of a Dog) (1971),",Yoram Kaniuk,Adam Resurrected,2009.0,https://www.imdb.com/title/tt0479341/
...,...,...,...,...,...
2086,"You're Best Alone (1943),",Norah Lofts,Guilt Is My Shadow,1950.0,https://www.imdb.com/title/tt0160336/
2087,"Your Ticket Is No Longer Valid (1977),",Romain Gary,Your Ticket Is No Longer Valid,1981.0,https://www.imdb.com/title/tt0080164/
2089,"Young Man of Manhattan (1930),",Katharine Brush,Young Man of Manhattan,1930.0,https://www.imdb.com/title/tt0021568/
2091,"The Story of Zarak Khan (1949),",A. J. Bevan,Zarak,1956.0,https://www.imdb.com/title/tt0051219/


In [7]:
#get imdb links for inner links of the movies
getwiki = pd.read_csv("a_to_z_with_links.csv")
getwiki = getwiki.dropna()
getwiki = getwiki[getwiki["imdb_links"].str.contains("wiki")]
inner_links = getwiki["imdb_links"].to_list()

imdb_links = []


for links in inner_links:
    try: 
        url ="https://en.wikipedia.org" + links
        # create soup
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        try:
            myuls = soup.find_all("ul")
            for uls in myuls:
                link_section = uls.find("a", href = True, class_="external text")
                if link_section:
                    imdb =  link_section["href"]
                    break
                else:
                    imdb = ""
        except Exception:
            pass 
    except Exception:
        pass
    imdb_links.append(imdb)
    print(imdb)

imdb_links

https://www.imdb.com/title/tt0094606/

https://www.imdb.com/title/tt0011313/
https://www.tcm.com/tcmdb/title/1526/enwp
http://movies.disney.com/the-adventures-of-huck-finn
https://www.imdb.com/title/tt0278332/
https://www.imdb.com/title/tt0278332/
https://www.imdb.com/title/tt0255651/
https://catalog.afi.com/Catalog/moviedetails/913
https://www.jstor.org/stable/43512826
https://en.wikipedia.org/w/index.php?title=Template:1960s-novel-stub&action=edit
https://catalog.afi.com/Catalog/moviedetails/26205
https://www.imdb.com/title/tt1823126/
https://books.google.com/books?id=naGQLGuiSC4C&dq
https://www.imdb.com/title/tt0072263/
https://www.imdb.com/title/tt0061075/
http://www.angelsanddemons.com/
https://www.imdb.com/title/tt0047834/
https://www.imdb.com/title/tt0075476/
https://en.wikipedia.org/wiki/Category:Love
https://en.wikipedia.org/w/index.php?title=Template:Dedham&action=edit
https://www.imdb.com/title/tt0081900/
https://www.imdb.com/title/tt0043293/
https://en.wikipedia.org/w/index

['https://www.imdb.com/title/tt0094606/',
 '',
 'https://www.imdb.com/title/tt0011313/',
 'https://www.tcm.com/tcmdb/title/1526/enwp',
 'http://movies.disney.com/the-adventures-of-huck-finn',
 'https://www.imdb.com/title/tt0278332/',
 'https://www.imdb.com/title/tt0278332/',
 'https://www.imdb.com/title/tt0255651/',
 'https://catalog.afi.com/Catalog/moviedetails/913',
 'https://www.jstor.org/stable/43512826',
 'https://en.wikipedia.org/w/index.php?title=Template:1960s-novel-stub&action=edit',
 'https://catalog.afi.com/Catalog/moviedetails/26205',
 'https://www.imdb.com/title/tt1823126/',
 'https://books.google.com/books?id=naGQLGuiSC4C&dq',
 'https://www.imdb.com/title/tt0072263/',
 'https://www.imdb.com/title/tt0061075/',
 'http://www.angelsanddemons.com/',
 'https://www.imdb.com/title/tt0047834/',
 'https://www.imdb.com/title/tt0075476/',
 'https://en.wikipedia.org/wiki/Category:Love',
 'https://en.wikipedia.org/w/index.php?title=Template:Dedham&action=edit',
 'https://www.imdb.com/t

In [15]:
getwiki["imdb_links"] = imdb_links
show =  pd.concat([show,getwiki])
show = show.dropna()
show = show[show["imdb_links"].str.contains('imdb')]
show

,books,authors,movies,released_year,imdb_links
0,"The A.B.C. Murders (1936),",Agatha Christie,The Alphabet Murders,1966.0,https://www.imdb.com/title/tt0060094/
2,"À ton image (1998),",Louise L. Lambrichs,À ton image,2004.0,https://www.imdb.com/title/tt0307343/
5,"About Schmidt (1996),",Louis Begley,About Schmidt,2002.0,https://www.imdb.com/title/tt0257360/
8,"Across 110th (1970),",Wally Ferris,Across 110th Street,1972.0,https://www.imdb.com/title/tt0068168/
9,"Adam Resurrected (Man, Son of a Dog) (1971),",Yoram Kaniuk,Adam Resurrected,2009.0,https://www.imdb.com/title/tt0479341/
...,...,...,...,...,...
2015,"The Winter of Our Discontent (1961),",John Steinbeck,The Winter of Our Discontent,1983.0,https://www.imdb.com/title/tt0086592/
2018,"The Witches of Eastwick (1984),",John Updike,The Witches of Eastwick,1987.0,https://www.imdb.com/title/tt0094332/
2057,"The Wonderful Wizard of Oz (1900),",L. Frank Baum,The Wiz,1978.0,https://www.imdb.com/title/tt0078504/
2062,"The Wonderful Wizard of Oz (1900),",L. Frank Baum,Lost in Oz,2002.0,https://www.imdb.com/title/tt4677846/


In [16]:
#split imdb id from imdb links
to_split =  show["imdb_links"].to_list()
imdb_id_list = []
for split in to_split:
    imdb_id = split.split("/")
    imdb_id_list.append(imdb_id[-2])
show["imdb_id"] = imdb_id_list
show = show.drop(columns=["authors", "released_year", "imdb_links"])
show.to_csv("books_to_movies(latest).csv", encoding="utf-8", index=False)
show

,books,movies,imdb_id
0,"The A.B.C. Murders (1936),",The Alphabet Murders,tt0060094
2,"À ton image (1998),",À ton image,tt0307343
5,"About Schmidt (1996),",About Schmidt,tt0257360
8,"Across 110th (1970),",Across 110th Street,tt0068168
9,"Adam Resurrected (Man, Son of a Dog) (1971),",Adam Resurrected,tt0479341
...,...,...,...
2015,"The Winter of Our Discontent (1961),",The Winter of Our Discontent,tt0086592
2018,"The Witches of Eastwick (1984),",The Witches of Eastwick,tt0094332
2057,"The Wonderful Wizard of Oz (1900),",The Wiz,tt0078504
2062,"The Wonderful Wizard of Oz (1900),",Lost in Oz,tt4677846


In [40]:
#split by regex test
test = "Adam Resurrected (Man, Son of a Dog) (1971)"
test2 = re.split(r" \([1-9]",test)
test2[0]

'Adam Resurrected (Man, Son of a Dog)'

In [46]:
#remove the date into book titles
book_list = show["books"].to_list()
new_books = []
for book in book_list:
    yearless_book = re.split(r" \([1-9]",book)
    new_books.append(yearless_book[0])

In [98]:
show["books"] = new_books
show

,books,movies,imdb_id,isbn
0,The A.B.C. Murders,The Alphabet Murders,tt0060094,
2,À ton image,À ton image,tt0307343,
5,About Schmidt,About Schmidt,tt0257360,
8,Across 110th,Across 110th Street,tt0068168,
9,"Adam Resurrected (Man, Son of a Dog)",Adam Resurrected,tt0479341,
...,...,...,...,...
2015,The Winter of Our Discontent,The Winter of Our Discontent,tt0086592,
2018,The Witches of Eastwick,The Witches of Eastwick,tt0094332,
2057,The Wonderful Wizard of Oz,The Wiz,tt0078504,
2062,The Wonderful Wizard of Oz,Lost in Oz,tt4677846,


In [5]:
#impoty kaggle's goodreads books
kaggle_books = pd.read_csv("kaggle_books.csv", error_bad_lines=False)
ktitle = kaggle_books["title"].to_list()
kisbn = kaggle_books["isbn"].to_list()
k_id = kaggle_books["bookID"].to_list()
kaggle_books

b'Skipping line 3350: expected 12 fields, saw 13\nSkipping line 4704: expected 12 fields, saw 13\nSkipping line 5879: expected 12 fields, saw 13\nSkipping line 8981: expected 12 fields, saw 13\n'


,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,11/1/2003,Scholastic
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,5/1/2004,Scholastic Inc.
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,41428,164,9/13/2004,Scholastic
...,...,...,...,...,...,...,...,...,...,...,...,...
11118,45631,Expelled from Eden: A William T. Vollmann Reader,William T. Vollmann/Larry McCaffery/Michael He...,4.06,1560254416,9781560254416,eng,512,156,20,12/21/2004,Da Capo Press
11119,45633,You Bright and Risen Angels,William T. Vollmann,4.08,0140110879,9780140110876,eng,635,783,56,12/1/1988,Penguin Books
11120,45634,The Ice-Shirt (Seven Dreams #1),William T. Vollmann,3.96,0140131965,9780140131963,eng,415,820,95,8/1/1993,Penguin Books
11121,45639,Poor People,William T. Vollmann,3.72,0060878827,9780060878825,eng,434,769,139,2/27/2007,Ecco


In [110]:
#jellyfish test
test1 = 'The A.B.C. Murders'
s = 'The A.B.C. Murders (Hercule Poirot  #13)'
jellytest = jellyfish.jaro_winkler_similarity(test1,s )
jellytest

0.89

In [131]:
#compare kaggle goodreads books against books to movies by title and get the book_id and isbn
isbn_list = []
books_id = [] 
print ("Starting ISBN and BookID check")
for title in new_books:
    for count in range(len(ktitle)-1):
        jelly_compare = jellyfish.jaro_winkler_similarity(title, ktitle[count])
        if jelly_compare > 0.85:
            pass_score = kisbn[count]
            book_id = k_id[count]
            break
        elif jelly_compare > 0.7 and jelly_compare < 0.85:
            pass_score = kisbn[count]
            book_id = k_id[count]
        elif jelly_compare > 0.65 and jelly_compare < 0.7:
            pass_score = kisbn[count]
            book_id = k_id[count]
        else:
            pass_score = "N/A"
            book_id = "N/A"
    books_id.append(book_id)    
    isbn_list.append(pass_score)
    pass_score = ""
    book_id = ""
print ("Check Done!")

Starting ISBN and BookID check
Check Done!


In [132]:
#add the book_id and isbn to the books to movies
show["isbn"] = isbn_list
show["book_id"] = books_id
show = show[["book_id","books","isbn","movies","imdb_id"]]
show.to_csv("books_to_movies(latest).csv", encoding="utf-8", index=False)
show

,book_id,books,isbn,movies,imdb_id
0,16322,The A.B.C. Murders,1579126243,The Alphabet Murders,tt0060094
2,N/A,À ton image,N/A,À ton image,tt0307343
5,N/A,About Schmidt,N/A,About Schmidt,tt0257360
8,N/A,Across 110th,N/A,Across 110th Street,tt0068168
9,N/A,"Adam Resurrected (Man, Son of a Dog)",N/A,Adam Resurrected,tt0479341
...,...,...,...,...,...
2015,2319,The Winter of Our Discontent,0440802504,The Winter of Our Discontent,tt0086592
2018,2319,The Witches of Eastwick,0440802504,The Witches of Eastwick,tt0094332
2057,9863,The Wonderful Wizard of Oz,0851703003,The Wiz,tt0078504
2062,9863,The Wonderful Wizard of Oz,0851703003,Lost in Oz,tt4677846


In [9]:
test = pd.read_csv("books_to_movies(latest).csv")
test = test.replace("N/A",np.nan)
test = test.dropna()
test = test.drop(columns="book_id")
test.to_csv("final_books_to_movies.csv", encoding="utf-8", index=False)